# Just Analysing the dataset

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [6]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data Preprocessing

In [8]:
df = pd.read_csv('BGdata.csv')
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
df.shape

(1599999, 6)

In [13]:
col_nm = ['Target' ,'ID','Date','Flag','User','Text']
df = pd.read_csv('BGdata.csv',names= col_nm,encoding = 'ISO-8859-1')

In [14]:
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
df.isnull().sum()

Target    0
ID        0
Date      0
Flag      0
User      0
Text      0
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Target  1600000 non-null  int64 
 1   ID      1600000 non-null  int64 
 2   Date    1600000 non-null  object
 3   Flag    1600000 non-null  object
 4   User    1600000 non-null  object
 5   Text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


Converting the 4 to 1 here

IF THE DATASET HAVE EVEN DISTRIBUTION OF DATA THEN WE NEED TO DO UPSAMPLING AND DOWNSAMPLING
IF NOT WE CAN DO THIS THINGS

In [18]:
df.Target.value_counts()

Target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
df.replace({"Target":{4:1}},inplace= True)

In [21]:
df.Target.value_counts()

Target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Comment 
1 --> Positive Comment

Stemming

Stemming is the process of reducing a word to its Root word 

EXAMPLE : Actor , Actress , Acting --> Act

In [30]:
port_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))


def Stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stop_words]
    stemmed_content = ' '.join(stemmed_content)  
    
    return stemmed_content

In [31]:
df['Stemmed_content'] = df['Text'].apply(Stemming)

In [32]:
df.head()

,Target,ID,Date,Flag,User,Text,Stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
X = df['Stemmed_content'].values
Y = df['Target'].values

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      shape=(1600000,), dtype=object)

In [36]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      shape=(1600000,), dtype=object)

In [37]:
Y

array([0, 0, 0, ..., 1, 1, 1], shape=(1600000,))

Splitting the data to training data and test data

In [94]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =0.2,stratify=Y,random_state=2)

In [95]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((1280000,), (320000,), (1280000,), (320000,))

In [96]:
# Converting the textual value to numerical values
v = TfidfVectorizer()
X_train = v.fit_transform(X_train)
X_test = v.fit_transform(X_test)

In [97]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453090 stored elements and shape (1280000, 461490)>
  Coords	Values
  (0, 436715)	0.27259876264838384
  (0, 354545)	0.3588091611460021
  (0, 185194)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235046)	0.41996827700291095
  (0, 443068)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407303)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406401)	0.32105459490875526
  (2, 433562)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443432)	0.3348599670252845
  (2, 266730)	0.24123230668976975
  (2, 409145)	0.15169282335109835
  (2, 178062)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288472)	0.16786949597862733
  (3, 406401)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318305)	0.21254698865277744
  (12

In [98]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2363913 stored elements and shape (320000, 179187)>
  Coords	Values
  (0, 104606)	0.4459147041194029
  (0, 51603)	0.25774294336856257
  (0, 107797)	0.17983918796121573
  (0, 158685)	0.3174206399662331
  (0, 26316)	0.2701195948137176
  (0, 163496)	0.180932348330391
  (0, 67046)	0.28383380524479984
  (0, 11837)	0.16338171192787918
  (0, 150603)	0.2214371376881977
  (0, 5650)	0.17295837060294847
  (0, 154815)	0.3501587145032445
  (0, 41330)	0.3621075009813407
  (0, 53904)	0.23662705066222536
  (1, 5650)	0.181974929607951
  (1, 2443)	0.2650067441796073
  (1, 99057)	0.2461083000957446
  (1, 141842)	0.21193862113578937
  (1, 134752)	0.4128221547743352
  (1, 83954)	0.35024490258566765
  (1, 56881)	0.48516752160641513
  (1, 136317)	0.5125311002730158
  (2, 71650)	0.5621198218335112
  (2, 99080)	0.2616858867134709
  (2, 13273)	0.3887964554215523
  (2, 8454)	0.35890991973427855
  :	:
  (319994, 65393)	0.4918118278116105
  (319995, 163

In [99]:
# Training the ML MODEL 

In [100]:
# LOGISTIC REGRESSION

In [101]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [102]:
# Accuracy score of the training data 

In [103]:
y_pred = model.predict(X_train)
training_data = accuracy_score(Y_train,y_pred)

In [104]:
training_data

0.79871875